# Querying the LiPDGraph database

For integration into ClimateDisk, we need to be able to query from the LinkedEarth GraphDB directly.

cfr was meant to work with a pickle version of the PAGES2k datasets. The new query needs to make the following assertion:
* Proxy that are sensitive to temperature
* Time between 0-2000 year CE
* Resolution of less than a year

In [1]:
import json
import requests
import pandas as pd
import io
import ast
import numpy as np

## Example pickle file

Let's first import the original pickled PAGES2k dataset to be used as a benchmark.

In [2]:
df_pickle = pd.read_pickle('data/pages2k_dataset.pkl')
df_pickle.head()

,paleoData_pages2kID,dataSetName,archiveType,geo_meanElev,geo_meanLat,geo_meanLon,year,yearUnits,paleoData_variableName,paleoData_units,paleoData_values,paleoData_proxy
0,NAm_153,NAm-Bennington.Luckman.2001,tree,1850.0,52.7,-118.3,"[1563.0, 1564.0, 1565.0, 1566.0, 1567.0, 1568....",AD,trsgi,NA,"[0.883, 0.942, 1.043, 0.945, 1.141, 0.715, 0.6...",TRW
1,Asi_245,Asi-SouthChina.Wang.1998,documents,2200.0,23.0,114.0,"[1500.0, 1510.0, 1520.0, 1530.0, 1540.0, 1550....",AD,temperature,index,"[0.157429639092533, -0.118646665464906, 0.3035...",historic
2,NAm_165,NAm-RedMountainPassSilverton.Graybill.1994,tree,3400.0,37.9,-107.7,"[1626.0, 1627.0, 1628.0, 1629.0, 1630.0, 1631....",AD,MXD,NA,"[1.071, 1.014, 1.014, 0.966, 1.005, 1.072, 1.0...",MXD
3,Asi_178,Asi-BURGPW.PAGES2k.2013,tree,2500.0,28.77,83.73,"[1303.0, 1304.0, 1305.0, 1306.0, 1307.0, 1308....",AD,trsgi,NA,"[0.889, 0.788, 0.836, 0.741, 0.613, 0.732, 0.7...",TRW
4,Asi_174,Asi-NEPA029.Krusic.2013,tree,4000.0,28.18,85.43,"[1559.0, 1560.0, 1561.0, 1562.0, 1563.0, 1564....",AD,trsgi,NA,"[1.172, 1.21, 1.145, 1.307, 1.368, 1.37, 1.152...",TRW


The number of datasets:

In [8]:
df_pickle.shape

(692, 12)

In [9]:
list(df_pickle.columns)

['paleoData_pages2kID',
 'dataSetName',
 'archiveType',
 'geo_meanElev',
 'geo_meanLat',
 'geo_meanLon',
 'year',
 'yearUnits',
 'paleoData_variableName',
 'paleoData_units',
 'paleoData_values',
 'paleoData_proxy']

## Using LiPDGraph

Let's first identify all the datasets that were part of PAGES2k, with variables tagged for "usedInGlobalTemperatureAnalysis":

In [8]:
url = 'https://linkedearth.graphdb.mint.isi.edu/repositories/LiPDVerse3'

query = """PREFIX le: <http://linked.earth/ontology#>
SELECT ?dsname ?val ?timeval ?archive ?pages2kID ?timeunits ?varname ?varunits ?lat ?lon ?alt ?proxysensor ?proxyobs 
FROM <http://linked.earth/lipd/Pages2k2_1_2> WHERE { 
	?ds le:includesPaleoData ?data .
    ?ds le:name ?dsname .
    ?ds le:proxyArchiveType ?archive .
    
    ?ds  le:collectedFrom ?loc .
    OPTIONAL { ?loc le:wgs84_Lat ?lat } .
    OPTIONAL { ?loc le:wgs84_Long ?lon } .
    OPTIONAL { ?loc le:wgs84_Alt ?alt } .

	?data le:foundInMeasurementTable ?table .
	?table le:includesVariable ?var .

    ?var le:name ?varname .     
    ?var le:useInGlobalTemperatureAnalysis true .
    ?var le:pages2kID ?pages2kID .    
    ?var le:hasValues ?val .    
    OPTIONAL { ?var le:hasUnits ?varunits } .
    OPTIONAL {
        ?var le:hasProxySystem ?proxysystem .
        ?proxysystem le:proxySensorType ?proxysensor .
        ?proxysystem le:proxyObservationType ?proxyobs .
    }    

    ?table le:includesVariable ?timevar .
    ?timevar le:name ?timevarname .
    VALUES ?timevarname {"year" "age"} .
    ?timevar le:hasValues ?timeval .
    ?timevar le:hasUnits ?timeunits .
}"""

response = requests.post(url, data = {'query': query})

data = io.StringIO(response.text)
df = pd.read_csv(data, sep=",")

df['val']=df['val'].apply(lambda row : json.loads(row) if isinstance(row, str) else row)
df['timeval']=df['timeval'].apply(lambda row : json.loads(row) if isinstance(row, str) else row)

df

,dsname,val,timeval,archive,pages2kID,timeunits,varname,varunits,lat,lon,alt,proxysensor,proxyobs
0,Ant-WDC05A.Steig.2013,"[-33.32873325, -35.6732, -33.1574, -34.2854, -...","[2005, 2004, 2003, 2002, 2001, 2000, 1999, 199...",glacier ice,Ant_006,AD,d18O,permil,-79.46,-112.09,1806.0,D18ODefaultSensor,D18O
1,NAm-MtLemon.Briffa.2002,"[0.968, 0.962, 1.013, 0.95, 1.008, 0.952, 1.02...","[1568, 1569, 1570, 1571, 1572, 1573, 1574, 157...",tree,NAm_107,AD,MXD,NaN,32.50,-110.80,2700.0,MXDDefaultSensor,MXD
2,Arc-Arjeplog.Bjorklund.2014,"[-0.829089212152348, -0.733882889924006, -0.89...","[1200, 1201, 1202, 1203, 1204, 1205, 1206, 120...",tree,Arc_065,AD,density,NaN,66.30,18.20,800.0,DeltaDensityDefaultSensor,DeltaDensity
3,Asi-CHIN019.Li.2010,"[1.465, 1.327, 1.202, 0.757, 1.094, 1.006, 1.2...","[1509, 1510, 1511, 1512, 1513, 1514, 1515, 151...",tree,Asi_041,AD,trsgi,NaN,29.15,99.93,2150.0,TRWDefaultSensor,TRW
4,NAm-Landslide.Luckman.2006,"[1.123, 0.841, 0.863, 1.209, 1.139, 1.056, 0.8...","[913, 914, 915, 916, 917, 918, 919, 920, 921, ...",tree,NAm_032,AD,trsgi,NaN,60.20,-138.50,800.0,TRWDefaultSensor,TRW
...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,NAm-ConroyLake.Gajewski.1988,"[17.1, 17.7, 17.2, 17.6, 17.3, 17.1, 17.7, 17....","[-17, -9, -1, 7, 15, 23, 31, 39, 46, 54, 62, 7...",lake sediment,NAm_068,BP,temperature,degC,46.30,-67.90,140.0,NaN,NaN
702,NAm-LittlePineLake.Gajewski.1988,"[20.2, 20.3, 20.1, 20.6, 19.7, 19.4, 19.4, 19....","[0, 16, 33, 49, 66, 82, 99, 115, 126, 136, 155...",lake sediment,NAm_078,BP,temperature,degC,45.30,-91.50,330.0,NaN,NaN
703,Ant-TaylorDome.Steig.2000,"[-38.98, -35.87, -39.19, nan, -38.85, -37.4, -...","[12.9000000000001, 15.2, 17.3, 19.5, 21.7, 23....",glacier ice,Ant_023,BP,d18O,permil,-77.78,158.72,2365.0,D18ODefaultSensor,D18O
704,Arc-LakeE.D'Andrea.2011,"[-0.624, -0.644, -0.625, -0.625, -0.581, -0.58...","[73.7964935794494, 84.5681478027345, 95.043389...",lake sediment,Arc_070,BP,temperature,degC,67.00,-50.70,170.0,NaN,NaN


## Comparison with the Pickle file

### Comparison by dataset name

Find the missing records:

In [9]:
missing=[]

for index,row in df_pickle.iterrows():
    if (row['dataSetName']) in df['dsname'].values:
        pass
    else:
        missing.append(row['dataSetName'])

missing

['Eur-CentralEurope.Dobrovolný.2009',
 'Arc-LakeHampträsk.Luoto.2009',
 'Ocn-Palmyra.Cobb.2003',
 'Eur-TatraMountains.Büntgen.2013',
 "Arc-BrayaSø.D'Andrea.2011",
 'Arc-LakeNataujärvi.Ojala.2005',
 'Eur-NorthernSpain.Martín-Chivelet.2011',
 'Eur-Lötschental.Büntgen.2006',
 'Arc-VøringPlateau.Berner.2011',
 'Ocn-PuntaMaroma.Vásquez-Bedoya.2012',
 'Eur-MaritimeFrenchAlps.Büntgen.2012',
 'Ocn-KNR140‐2‐59GGC.Saenger.2011',
 'Arc-Hvítárvatn.Larsen.2011',
 'Ocn-JacafFjord.Sepúlveda.2009',
 'NAm-AthabascaGlacier.Schweingruber.1996',
 'Eur-EuropeanAlps.Büntgen.2011']

In [10]:
reverse_missing=[]

for index,row in df.iterrows():
    if (row['dsname']) in df_pickle['dataSetName'].values:
        pass
    else:
        reverse_missing.append(row['dsname'])

reverse_missing

['Arc-LakeNataujaervi.Ojala.2005',
 'Eur-CentralEurope.Dobrovolny.2009',
 'Eur-TatraMountains.Buentgen.2013',
 'Eur-MaritimeFrenchAlps.Buentgen.2012',
 'Ocn-JacafFjord.Sepulveda.2009',
 'Ocn-Palmyra.Cobb.2013',
 'Eur-EuropeanAlps.Buentgen.2011',
 'Arc-Hvitarvatn.Larsen.2011',
 "Arc-BrayaSoe.D'Andrea.2011",
 'Eur-Loetschental.Buentgen.2006',
 'Eur-NorthernSpain.Martin-Chivelet.2011',
 'Arc-VoeringPlateau.Berner.2011',
 'Arc-LakeHamptraesk.Luoto.2009',
 'Ocn-PuntaMaroma.Vasquez-Bedoya.2012']

### Comparison by pages2k ID

In [11]:
missing_id=[]

for index,row in df_pickle.iterrows():
    if (row['paleoData_pages2kID']) in df['pages2kID'].values:
        pass
    else:
        missing_id.append([row['paleoData_pages2kID'], row['dataSetName']])

missing_id

[['Ocn_028', 'Ocn-KNR140‐2‐59GGC.Saenger.2011'],
 ['NAm_131', 'NAm-AthabascaGlacier.Schweingruber.1996']]

In [12]:
reverse_missing_id=[]

for index,row in df.iterrows():
    if (row['pages2kID']) in df_pickle['paleoData_pages2kID'].values:
        pass
    else:
        missing_id.append(row['pages2kID'])

reverse_missing_id

[]

In [14]:
import lipd as lpd

In [16]:
D=lpd.readLipd('/Users/deborahkhider/Documents/LinkedEarth/LiPD/Pages2kTemperature2_1_2/NAm-Athabasca.Schweingruber.1996.lpd')

Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: NAm-Athabasca.Schweingruber.1996.lpd
Finished read: 1 record


In [17]:
D.keys()

dict_keys(['paleoData', 'archiveType', 'createdBy', 'dataContributor', 'dataSetName', 'googleDataURL', 'googleMetadataWorksheet', 'googleSpreadSheetKey', 'originalDataURL', 'tagMD5', 'datasetId', 'maxYear', 'minYear', 'changelog', 'pub', 'geo', '@context', 'lipdVersion'])

In [18]:
D['dataSetName']

'NAm-Athabasca.Schweingruber.1996'

In [19]:
ts_list=lpd.extractTs(D)

extracting paleoData...
extracting: NAm-Athabasca.Schweingruber.1996
Created time series: 26 entries


In [49]:
for item in ts_list:
    if 'paleoData_pages2kID' in item.keys():
        print(item['paleoData_pages2kID'] + ',' + item['paleoData_variableName'] + ',' + str(item['geo_meanLon'])
              + ',' + str(item['geo_meanLat']) +','+str(item['geo_meanElev']))

X-NAm_025x,trsgi,-117.3,51.4,2000.0
NAm_026,MXD,-117.3,51.4,2000.0


In [43]:
df_pickle[df_pickle['dataSetName']=='NAm-AthabascaGlacier.Schweingruber.1996']

,paleoData_pages2kID,dataSetName,archiveType,geo_meanElev,geo_meanLat,geo_meanLon,year,yearUnits,paleoData_variableName,paleoData_units,paleoData_values,paleoData_proxy
548,NAm_131,NAm-AthabascaGlacier.Schweingruber.1996,tree,2000.0,52.3,-117.3,"[1665.0, 1666.0, 1667.0, 1668.0, 1669.0, 1670....",AD,MXD,NA,"[1.051, 1.09, 1.022, 1.032, 1.042, 1.003, 1.03...",MXD


In [51]:
df_pickle[df_pickle['paleoData_pages2kID']=='NAm_026']

,paleoData_pages2kID,dataSetName,archiveType,geo_meanElev,geo_meanLat,geo_meanLon,year,yearUnits,paleoData_variableName,paleoData_units,paleoData_values,paleoData_proxy
335,NAm_026,NAm-Athabasca.Schweingruber.1996,tree,2000.0,51.4,-117.3,"[1072.0, 1073.0, 1074.0, 1075.0, 1076.0, 1077....",AD,MXD,NA,"[0.968, 1.058, 1.093, 0.996, 1.058, 0.996, 0.9...",MXD


In [54]:
df[df['pages2kID']=='NAm_026']

,dsname,val,timeval,archive,pages2kID,timeunits,varname,varunits,lat,lon,alt,proxysensor,proxyobs
211,NAm-Athabasca.Schweingruber.1996,"[0.968, 1.058, 1.093, 0.996, 1.058, 0.996, 0.9...","[1072, 1073, 1074, 1075, 1076, 1077, 1078, 107...",tree,NAm_026,AD,MXD,NaN,51.4,-117.3,2000.0,MXDDefaultSensor,MXD


In [55]:
D2 = lpd.readLipd('/Users/deborahkhider/Documents/LinkedEarth/LiPD/Pages2kTemperature2_1_2/Ocn-KNR140‐2‐59GGC.Saenger.2011.lpd')

Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Ocn-KNR140‐2‐59GGC.Saenger.2011.lpd
Error: merge_csv_table: paleo, 'Ocn-KNR140‐2‐59GGC.Saenger.2011.paleo1measurement1.csv'
Error: merge_csv_table: paleo, 'Ocn-KNR140‐2‐59GGC.Saenger.2011.paleo2measurement1.csv'
Error: merge_csv_table: chron, 'Ocn-KNR140‐2‐59GGC.Saenger.2011.chron1measurement1.csv'
Error: get_csv_from_columns: Ocn-KNR140‐2‐59GGC.Saenger.2011.paleo0measurement0.csv, 'values'
Error: get_csv_from_columns: Ocn-KNR140‐2‐59GGC.Saenger.2011.paleo1measurement0.csv, 'values'
Error: get_csv_from_columns: Ocn-KNR140‐2‐59GGC.Saenger.2011.chron0measurement0.csv, 'values'
Finished read: 1 record


In [56]:
D2

{'paleoData': OrderedDict([('paleo0',
               OrderedDict([('measurementTable',
                             OrderedDict([('paleo0measurement0',
                                           {'googleWorkSheetKey': 'o7w7w6o',
                                            'tableName': 'paleo0measurement0',
                                            'missingValue': 'NaN',
                                            'columns': OrderedDict([('temperature',
                                                          {'paleoDataTableName': 'DataQC',
                                                           'paleoDataMD5': '5b703ada672646a7dce35e276b0dfd4a',
                                                           'measurementTableName': 'measurementTable1',
                                                           'measurementTableMD5': '8ed99f1c18d48681d33ba77aa22cec55',
                                                           'QCCertification': 'KT, CS, HVM, BM, JAA',
               

### LiPDGraph v3

Using the third version of the LiPDGraph (circa May 2023), and look for data in PAGES2k and ISO2k, that are proxies and interpreted as temperature. This may require filtering on a DataFrame after the original query. 

In [2]:
import json
import requests
import pandas as pd
import io
import ast
import numpy as np

In [3]:
url = 'https://linkedearth.graphdb.mint.isi.edu/repositories/LiPDVerse3'

query = """PREFIX le: <http://linked.earth/ontology#>
PREFIX wgs84: <http://www.w3.org/2003/01/geo/wgs84_pos#>


SELECT ?dataSetName ?archiveType ?geo_meanLat ?geo_meanLon ?geo_meanElev 
    ?paleoData_variableName ?paleoData_values ?paleoData_units 
    ?paleoData_proxy ?paleoData_proxyGeneral ?time_variableName ?time_values 
	?time_units ?compilationName ?TSID where{
    
    ?ds a le:Dataset .
    ?ds le:name ?dataSetName .
    OPTIONAL{?ds le:proxyArchiveType ?archiveType .}
    
    ?ds le:collectedFrom ?loc .
    ?loc wgs84:lat ?geo_meanLat .
    ?loc wgs84:long ?geo_meanLon .
    OPTIONAL {?loc wgs84:alt ?geo_meanElev .}
    
    ?ds le:includesPaleoData ?data .
    ?data le:foundInMeasurementTable ?table .
    ?table le:includesVariable ?var . 
    ?var le:name ?paleoData_variableName .
    ?var le:hasValues ?paleoData_values .
    OPTIONAL{?var le:hasUnits ?paleoData_units .}
    OPTIONAL{?var le:proxy ?paleoData_proxy .}
    OPTIONAL{?var le:proxyGeneral ?paleoData_proxyGeneral .}
    ?var le:partOfCompilation ?compilation . 
    ?compilation le:name ?compilationName .
    VALUES ?compilationName {"iso2k" "Pages2kTemperature"} .
    ?var le:useInGlobalTemperatureAnalysis True .
    OPTIONAL{?var le:hasVariableID ?TSID} .
    
    
    ?table le:includesVariable ?timevar .
    ?timevar le:name ?time_variableName .
    FILTER (regex(?time_variableName, "year.*")).
    ?timevar le:hasValues ?time_values .
    OPTIONAL{?timevar le:hasUnits ?time_units .}
    
}"""

In [4]:
response = requests.post(url, data = {'query': query})

data = io.StringIO(response.text)
df = pd.read_csv(data, sep=",")

df['paleoData_values']=df['paleoData_values'].apply(lambda row : json.loads(row) if isinstance(row, str) else row)
df['time_values']=df['time_values'].apply(lambda row : json.loads(row) if isinstance(row, str) else row)

df

,dataSetName,archiveType,geo_meanLat,geo_meanLon,geo_meanElev,paleoData_variableName,paleoData_values,paleoData_units,paleoData_proxy,paleoData_proxyGeneral,time_variableName,time_values,time_units,compilationName,TSID
0,LS16STCL,LakeSediment,50.830,-116.390,1126.0,d18O,"[-7.81, -5.91, -9.03, -5.35, -5.61, -5.98, -5....",permil,d18O,NaN,year,"[2009.0, 2008.3, 2007.8, 2007.4, 2007.0, 2006....",AD,iso2k,LPD7dc5b9ba
1,CO00URMA,Coral,0.933,173.000,6.0,d18O,"[-4.8011, -4.725, -4.6994, -4.86, -5.0886, -5....",permil,d18O,NaN,year,"[1994.5, 1994.33, 1994.17, 1994.0, 1993.83, 19...",AD,iso2k,Ocean2kHR_177
2,CO00URMA,Coral,0.933,173.000,6.0,d18O,"[-4.8011, -4.725, -4.6994, -4.86, -5.0886, -5....",permil,d18O,NaN,year,"[1994.5, 1994.33, 1994.17, 1994.0, 1993.83, 19...",AD,iso2k,Ocean2kHR_177
3,CO00URMA,Coral,0.933,173.000,6.0,d18O,"[-4.8011, -4.725, -4.6994, -4.86, -5.0886, -5....",permil,d18O,NaN,year,"[1994.5, 1994.33, 1994.17, 1994.0, 1993.83, 19...",AD,iso2k,Ocean2kHR_177
4,CO05KUBE,Coral,32.467,-64.700,-12.0,d18O,"[-4.15, -3.66, -3.69, -4.07, -3.95, -4.12, -3....",permil,d18O,NaN,year,"[1983.21, 1983.13, 1983.04, 1982.96, 1982.88, ...",AD,iso2k,Ocean2kHR_105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,Asi-PAKI001.Esper.2007,tree,36.030,74.580,3200.0,trsgi,"[1.143, 1.368, 1.062, 1.465, 1.448, 1.174, 0.9...",NaN,TRW,NaN,year,"[1593, 1594, 1595, 1596, 1597, 1598, 1599, 160...",AD,Pages2kTemperature,Asia_186
652,Asi-PAKI014.Esper.2007,tree,35.170,75.500,3100.0,trsgi,"[1.282, 1.244, 1.457, 1.33, 1.346, 1.149, 1.07...",NaN,TRW,NaN,year,"[1412, 1413, 1414, 1415, 1416, 1417, 1418, 141...",AD,Pages2kTemperature,Asia_210
653,Ocn-TongueoftheOcean.Rosenheim.2005,sclerosponge,23.504,-76.577,-143.0,Sr_Ca,"[10.884, 10.598, 10.544, 10.422, 10.26, 10.559...",mmol/mol,Sr/Ca,NaN,year,"[1890, 1891, 1892, 1893, 1894, 1895, 1896, 189...",AD,Pages2kTemperature,Ocean2kHR_115
654,Ocn-TongueoftheOcean.Rosenheim.2005,sclerosponge,23.504,-76.577,-143.0,d18O,"[0.356, -0.053, 0.639, 0.714, 0.6, 0.586, 0.58...",permil,d18O,NaN,year,"[1890, 1891, 1892, 1893, 1894, 1895, 1896, 189...",AD,Pages2kTemperature,Ocean2kHR_114


In [5]:
df['paleoData_variableName'].unique()

array(['d18O', 'd2H', 'MXD', 'density', 'trsgi', 'Sr_Ca', 'temperature',
       'X_radiograph_dark_layer', 'composite', 'thickness', 'RABD660_670',
       'calcification', 'dD'], dtype=object)

In [13]:
df1 = df[['archiveType','paleoData_variableName']]

df1.groupby(['archiveType','paleoData_variableName'], as_index=False).value_counts()

,archiveType,paleoData_variableName,count
0,Coral,d18O,47
1,GlacierIce,d18O,1
2,LakeSediment,d18O,3
3,LakeSediment,d2H,1
4,Sclerosponge,d18O,3
5,bivalve,d18O,1
6,borehole,temperature,2
7,coral,Sr_Ca,24
8,coral,calcification,2
9,coral,composite,6
